In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from keras.layers import Flatten, Input
from keras.models import Model
from keras.applications.vgg16 import VGG16
import glob
import cv2
import os
import numpy as np

input = Input(shape=(224, 224, 3), name = 'image_input')
# output dimension for VGG16 is a tensor of 7 x 7 x 512
vgg_model = VGG16(weights='imagenet', include_top=False)

our_feats = vgg_model(input)
our_feats = Flatten()(our_feats)

model = Model(inputs=input, outputs=our_feats)
model.summary() # See our model

chars_path = ["L/224", "O/224", "C/224"]
drive_prefix = "/content/gdrive/My Drive/uns/S2P1/computer_vision/Letters"

for char_p in chars_path:
  path_to_images = os.path.join(drive_prefix, char_p)
  image_list = sorted(glob.glob(path_to_images + "/*"))
  print("Number of images for {}: {}".format(char_p, len(image_list)))
  X = np.empty((len(image_list), 224, 224, 3))
  for i, image_path in enumerate(image_list):
      image = cv2.imread(image_path)
      image_resized = cv2.resize(image, (224, 224))
      X[i] = image_resized

  output_feats = model.predict(X) # Extract features of our images

  print(len(output_feats))
  character = char_p.split("/")[0]
  print("Saving features for the character: {}".format(character))
  with open(drive_prefix + '/letters_vgg_features.txt', 'a') as f:
    for fea in output_feats:
      f.write("{},".format(character))
      f.write(",".join(fea.astype(str)))
      f.write("\n")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_11 (Flatten)         (None, 25088)             0         
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________
Number of images for L/224: 175
175
Saving features for the character: L
Number of images for O/224: 91
91
Saving features for the character: O
Number of images for C/224: 104
104
Saving features for the character: C
